<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica
</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 05. Obtener lista de hospitales georeferenciados

* Existen dos fuentes:
    - Catálogo Nacional de Hospitales del SNS (sin referenciar). Archivo Hospitales_de_Espana.csv
    - Hospitales georeferenciados en ERSI. Archivo CNH_2022.xlsx

* Para tener los hospitales georeferenciados se combinan estas fuentes 

* Los datos se han descargado a mano y se han copiado a la carpeta `Data/Input` desde los siguientes enlaces:
    - SNS: https://www.sanidad.gob.es/ciudadanos/prestaciones/centrosServiciosSNS/hospitales/home.htm
    - ESRI: https://opendata.esri.es/datasets/ComunidadSIG::hospitales-de-españa/explore?location=39.565169%2C-5.955525%2C6.06


NOTA: por motivos de eficiencia se realizan en este notebooks tareas propias de otros proceso del dominio II, en particular el proceso 06 _Data Adequacy_

Adicionalmente: se actualizan los nombres de hospitales de las listas de espera para que sean consistentes con la geolocalización y el catálogo nacional.







<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

### Packages to use

_ELIMINAR O AÑADIR LO QUE TOQUE. COPIAR VERSIONES AL FINAL Y QUITAR CÓDIGO DE VERSIONES_

- {tcltk} para selección interactiva de archivos locales
- {sf} para trabajar con georeferenciación
- {readr} para leer y escribir archivos csv
- {dplyr} para explorar datos
- {stringr} para manipulación de cadenas de caracteres
- {tidyr} para organización de datos


In [2]:
library(sf)
library(readxl)
library(readr)
library(dplyr)
library(stringr)

p <- c("tcltk", "sf", "readr", "dplyr", "stringr", "readxl")


Linking to GEOS 3.10.2, GDAL 3.4.2, PROJ 8.2.1; sf_use_s2() is TRUE


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Paths

In [3]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

1. Hospitales geolocalizados en ESRI

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [4]:
# file_data_01 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [5]:
iFile_01 <- "Hospitales_de_Espana.csv"
file_data_01 <- paste0(iPath, iFile_01)

if(file.exists(file_data_01)){
    cat("Se leerán datos del archivo: ", file_data_01)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/Hospitales_de_Espana.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [6]:
data_01 <- read_csv(file_data_01)


Rows: 924 Columns: 47
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (24): CODCNH, NOMBRE, DIRECCION, CODMU, MUNICIPIOS, CODPROV, PROVINCIAS,...
dbl (23): OBJECTID, TELEFONO, TELEFONO2, TELEFAX, NCAMAS, CODFI, CODPAT, COD...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [7]:
data_01 |> glimpse()


Rows: 924
Columns: 47
$ OBJECTID                <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…
$ CODCNH                  <chr> "010035", "010040", "010053", "010066", "01008…
$ NOMBRE                  <chr> "RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUI…
$ DIRECCION               <chr> "ÁLAVA, 43", "BEATO TOMÁS DE ZUMÁRRAGA, 10", "…
$ TELEFONO                <dbl> 945006555, 945140900, 945252500, 945142100, 94…
$ TELEFONO2               <dbl> NA, NA, NA, NA, 945006876, 945007007, 96759717…
$ TELEFAX                 <dbl> 945006587, 945145709, 945279260, 945143318, 94…
$ CODMU                   <chr> "010590", "010590", "010590", "010590", "01031…
$ MUNICIPIOS              <chr> "Vitoria-Gasteiz", "Vitoria-Gasteiz", "Vitoria…
$ CODPROV                 <chr> "01", "01", "01", "01", "01", "01", "02", "02"…
$ PROVINCIAS              <chr> "ÁLAVA", "ÁLAVA", "ÁLAVA", "ÁLAVA", "ÁLAVA", "…
$ CODAUTO                 <chr> "16", "16", "16", "16", "16", "16", "08", "08"…
$ COMUNIDADES     

Muestra de datos:

In [8]:
data_01 |> slice_head(n = 5)

OBJECTID,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,MUNICIPIOS,CODPROV,...,BCO,ALI,SPECT,PET,MAMOS,DO,DIAL,X,Y,CalidadGeocodificacion
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,010035,RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIÁTRICO DE ARABA),"ÁLAVA, 43",945006555,NA,945006587,010590,Vitoria-Gasteiz,01,...,0,0,0,0,0,0,0,-2.678612,42.83566,Manual
2,010040,HOSPITAL SAN JOSÉ,"BEATO TOMÁS DE ZUMÁRRAGA, 10",945140900,NA,945145709,010590,Vitoria-Gasteiz,01,...,0,0,0,0,1,1,1,-2.676640,42.84966,PointAddress
3,010053,HOSPITAL QUIRÓNSALUD VITORIA,"ESPERANZA, 3",945252500,NA,945279260,010590,Vitoria-Gasteiz,01,...,0,0,0,0,1,1,0,-2.668035,42.84976,PointAddress
4,010066,"HOSPITAL DE CUIDADOS SAN ONOFRE, S.L. (HOSPITAL DE CUIDADOS SAN ONOFRE)","SALVATIERRABIDE, 9",945142100,NA,945143318,010590,Vitoria-Gasteiz,01,...,0,0,0,0,0,0,0,-2.680386,42.83758,PointAddress
5,010088,HOSPITAL DE LEZA,"CTRA. VITORIA - LOGROÑO, KM. 59",945006900,945006876,945006901,010318,Laguardia,01,...,0,0,0,0,0,0,0,-2.638635,42.57599,Manual


2. Hospitales del SNS

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [9]:
# file_data_02 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [10]:
iFile_02 <- "CNH_2022.xlsx"
file_data_02 <- paste0(iPath, iFile_02)

if(file.exists(file_data_02)){
    cat("Se leerán datos del archivo: ", file_data_02)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CNH_2022.xlsx

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [11]:
data_02 <- read_excel(file_data_02, sheet = "DIRECTORIO DE HOSPITALES")


Estructura de  los datos:

In [12]:
data_02 |> glimpse()

Rows: 832
Columns: 22
$ CCN                     <chr> "1531000730", "1531000737", "1531000733", "153…
$ CODCNH                  <chr> "310121", "310044", "310060", "310057", "31007…
$ `Nombre Centro`         <chr> "HOSPITAL GARCIA ORCOYEN", "HOSPITAL SAN JUAN …
$ Dirección               <chr> "CALLE SANTA SORIA 22 -", "CALLE BELOSO ALTO 3…
$ Teléfono                <chr> "848435000", "948231800", "948255400", "948140…
$ `Cód. Municipio`        <chr> "310977", "312016", "312016", "312016", "31201…
$ Municipio               <chr> "Estella-Lizarra", "Pamplona/Iruña", "Pamplona…
$ `Cód. Provincia`        <chr> "31", "31", "31", "31", "31", "31", "31", "31"…
$ Provincia               <chr> "NAVARRA", "NAVARRA", "NAVARRA", "NAVARRA", "N…
$ `Cód. CCAA`             <chr> "15", "15", "15", "15", "15", "15", "15", "15"…
$ CCAA                    <chr> "C. FORAL DE NAVARRA", "C. FORAL DE NAVARRA", …
$ `Código Postal`         <chr> "31200", "31006", "31008", "31014", "31006", "…
$ CAMAS           

Muestra de datos:

In [13]:
data_02 |> slice_head(n = 5)

CCN,CODCNH,Nombre Centro,Dirección,Teléfono,Cód. Municipio,Municipio,Cód. Provincia,Provincia,Cód. CCAA,...,CAMAS,Cód. Clase de Centro,Clase de Centro,Cód. Dep. Funcional,Dependencia Funcional,Forma parte Complejo,CODIDCOM,Nombre del Complejo,ALTA,Email
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1531000730,310121,HOSPITAL GARCIA ORCOYEN,CALLE SANTA SORIA 22 -,848435000,310977,Estella-Lizarra,31,NAVARRA,15,...,106,C11,Hospitales Generales,2,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNIDADES AUTÓNOMAS,N,NA,NA,N,GERENCIA.AREASALUD.ESTELLA@NAVARRA.ES
1531000737,310044,HOSPITAL SAN JUAN DE DIOS,CALLE BELOSO ALTO 3 -,948231800,312016,Pamplona/Iruña,31,NAVARRA,15,...,256,C11,Hospitales Generales,20,PRIVADOS,N,NA,NA,N,HSJDPAMPLONA@OHSJD.ES
1531000733,310060,CLINICA UNIVERSIDAD DE NAVARRA,AVENIDA PIO XII 36 -,948255400,312016,Pamplona/Iruña,31,NAVARRA,15,...,230,C11,Hospitales Generales,20,PRIVADOS,N,NA,NA,N,DIRECCIONCUN@UNAV.ES
1531000736,310057,CLINICA PSIQUIATRICA PADRE MENNI,CALLE JOAQUIN BEUNZA 45 -,948140611,312016,Pamplona/Iruña,31,NAVARRA,15,...,237,C14,Hospitales de salud mental y tratamiento de toxicomanías,20,PRIVADOS,N,NA,NA,N,GERENCIA.MENNIPAMPLONA@HOSPITALARIAS.ES
1531000738,310076,CLINICA ARCANGEL SAN MIGUEL - PAMPLONA,CALLE BELOSO ALTO. 32 -,948296000,312016,Pamplona/Iruña,31,NAVARRA,15,...,72,C11,Hospitales Generales,20,PRIVADOS,N,NA,NA,N,CLINICASANMIGUEL@IMQNAVARRA.COM


3. Lista de espera

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [14]:
# file_data_03 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [15]:
iFile_03 <- "CU_25_05_04_lista_espera.csv"
file_data_03 <- paste0(iPath, iFile_03)

if(file.exists(file_data_03)){
    cat("Se leerán datos del archivo: ", file_data_03)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_05_04_lista_espera.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [16]:
data_03 <- read_csv(file_data_03)


Rows: 55680 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Hospital, Especialidad
dbl (4): total_pacientes, media_tiempo_dias, ano, semana

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [17]:
data_03 |> glimpse()

Rows: 55,680
Columns: 6
$ Hospital          <chr> "HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA A…
$ Especialidad      <chr> "Angiología y Cirugía Vascular", "Angiología y Cirug…
$ total_pacientes   <dbl> 439, 437, 429, 419, 418, NA, 408, 399, 396, 393, 378…
$ media_tiempo_dias <dbl> 56.29, 55.13, 53.99, 56.29, 55.82, NA, 57.45, 57.61,…
$ ano               <dbl> 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022…
$ semana            <dbl> 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, …


Muestra de datos:

In [18]:
data_03 |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,media_tiempo_dias,ano,semana
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,439,56.29,2022,44
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,437,55.13,2022,43
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,429,53.99,2022,42
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,419,56.29,2022,41
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,418,55.82,2022,40


4. Áreas sanitarias

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [19]:
# file_data_04 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [20]:
iFile_04 <- "CU_25_05_03_areasgeo.json"
file_data_04 <- paste0(iPath, iFile_04)

if(file.exists(file_data_04)){
    cat("Se leerán datos del archivo: ", file_data_04)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_05_03_areasgeo.json

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [21]:
data_04 <- st_read(file_data_04, 
    options = "ENCODING=latin1") |> 
    st_set_crs(23030) |> 
    st_transform(4326)


options:        ENCODING=latin1 


Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Message 6: driver GeoJSON does not support open option ENCODING”


Reading layer `CU_25_05_03_areasgeo' from data source 
  `/Users/emilio.lcano/academico/gh_repos/__transferencia/citizenlab/CitizenLab-Research-and-Development/casos_urjc/notebooks/II_data_processing/25_listas_espera/Data/Input/CU_25_05_03_areasgeo.json' 
  using driver `GeoJSON'
Simple feature collection with 11 features and 3 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 365635.7 ymin: 4415272 xmax: 495484 ymax: 4557310
Geodetic CRS:  WGS 84


Warning message:
“st_crs<- : replacing crs does not reproject data; use st_transform for that”


Estructura de  los datos:

In [22]:
data_04 |> glimpse()

Rows: 11
Columns: 4
$ CODBDT    <int> 958285, 958286, 958287, 958288, 958289, 958290, 958291, 9582…
$ GEOCODIGO <chr> "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", …
$ DESBDT    <chr> "Sur-Este", "Centro-Norte", "Este", "Noreste", "Norte", "Oes…
$ geometry  <MULTIPOLYGON [°]> MULTIPOLYGON (((-3.423161 4..., MULTIPOLYGON ((…


Muestra de datos:

In [23]:
data_04 |> tibble() |> slice_head(n = 5)

CODBDT,GEOCODIGO,DESBDT,geometry
<int>,<chr>,<chr>,<MULTIPOLYGON [°]>
958285,01,Sur-Este,MULTIPOLYGON (((-3.423161 4...
958286,02,Centro-Norte,MULTIPOLYGON (((-3.499207 4...
958287,03,Este,MULTIPOLYGON (((-3.323953 4...
958288,04,Noreste,MULTIPOLYGON (((-3.653662 4...
958289,05,Norte,MULTIPOLYGON (((-3.537981 4...


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Catálogo nacional de hospitales
* Geolocalización hospitales en datos abiertos ESRI

Incluir apartados si procede para: Extracción de datos (select, filter), Transformación de datos, (mutate, joins, ...). Si es necesario tratar datos perdidos, indicarlo también en NB 09.2

Si no aplica: Estos datos no requieren tareas de este tipo.

#### Data transform

- Los datos del Excel del ministerio no tienen tildes
- Los datos de listas de espera de la CM no tienen tildes
- Los datos de ESRI tienen tildes

Quitamos las tildes del archivo de ESRI

In [24]:
tdata_01 <- data_01 |>
    mutate(NOMBRE = chartr("ÁÉÍÓÚ", "AEIOU", NOMBRE))

Análisis inconsistencias en nombres

Hospitales cuyos nombres no coinciden:

In [25]:
unique(data_03$Hospital)[!unique(data_03$Hospital) %in% tdata_01$NOMBRE] |> 
  sort()

[1] "HOSPITAL DE EL ESCORIAL"                   
[2] "HOSPITAL FUNDACION JIMENEZ DIAZ"           
[3] "HOSPITAL UNIVERSITARIO GENERAL DE VILLALBA"
[4] "HOSPITAL UNIVERSITARIO RAMON Y CAJAL"      
[5] "HOSPITAL UNIVERSITARIO REY JUAN CARLOS"

Alternativas:

In [26]:
tdata_01 |> 
  filter(str_detect(NOMBRE, "ESCORIAL|VILLALBA|JUAN CARLOS|JIMENEZ DIAZ|RAMON Y CAJAL")) |> 
  arrange(NOMBRE) |> 
  pull(NOMBRE)

[1] "HOSPITAL EL ESCORIAL"                         
[2] "HOSPITAL GENERAL DE VILLALBA"                 
[3] "HOSPITAL RAMON Y CAJAL"                       
[4] "HOSPITAL REY JUAN CARLOS"                     
[5] "HOSPITAL UNIVERSITARIO FUNDACION JIMENEZ DIAZ"

- Los datos de ESRI contienen el código de hospital de CNH
- El nombre definitivo debería ser el de la base de datos oficial de CNH
- Asignamos nombres de ESRI provisionalmente a los de listas de espera

In [27]:
tdata_03 <- data_03 |> 
  mutate(Hospital = recode(Hospital, 
                           "HOSPITAL DE EL ESCORIAL" = "HOSPITAL EL ESCORIAL",
                           "HOSPITAL FUNDACION JIMENEZ DIAZ" = "HOSPITAL UNIVERSITARIO FUNDACION JIMENEZ DIAZ",
                           "HOSPITAL UNIVERSITARIO GENERAL DE VILLALBA" = "HOSPITAL GENERAL DE VILLALBA",
                           "HOSPITAL UNIVERSITARIO RAMON Y CAJAL" = "HOSPITAL RAMON Y CAJAL",
                           "HOSPITAL UNIVERSITARIO REY JUAN CARLOS" = "HOSPITAL REY JUAN CARLOS"))

* Asignar coordendas a catálogo de hospitales que están en la Comunidad de Madrid
* Convertir número de camas en numérica

In [28]:
tdata_02 <- data_02 |> 
  mutate(CAMAS = as.numeric(CAMAS)) |> 
  filter(`Cód. Provincia` == "28") |> 
  inner_join(tdata_01 |> 
              select(-NCAMAS, -CODIDCOM, -ALTA), 
            by = "CODCNH")

In [29]:
glimpse(tdata_02)

Rows: 87
Columns: 65
$ CCN                     <chr> "1328020932", "1328000034", "1328000070", "132…
$ CODCNH                  <chr> "281371", "280745", "281124", "281071", "28129…
$ `Nombre Centro`         <chr> "HOSPITAL HM VALLES", "HOSPITAL UNIVERSITARIO …
$ Dirección               <chr> "CALLE SANTIAGO 14", "CARRETERA DE MECO S/N", …
$ Teléfono                <chr> "918883262", "918878100", "917810682", "916219…
$ `Cód. Municipio`        <chr> "280053", "280053", "280072", "280072", "28013…
$ Municipio               <chr> "Alcalá de Henares", "Alcalá de Henares", "Alc…
$ `Cód. Provincia`        <chr> "28", "28", "28", "28", "28", "28", "28", "28"…
$ Provincia               <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRI…
$ `Cód. CCAA`             <chr> "13", "13", "13", "13", "13", "13", "13", "13"…
$ CCAA                    <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRI…
$ `Código Postal`         <chr> "28801", "28805", "28922", "28922", "28300", "…
$ CAMAS            

Asignación del área sanitaria a los hospitales. Primero usamos objetos sf para hacer el join por geometrías, y después eliminamos estas para tener un data.frame normal y guardar como csv. 

In [30]:
tdata_01_sf <- tdata_02 |> st_as_sf(coords = c("X", "Y")) |> st_set_crs(4326) 

tdata_01_area <- tdata_01_sf |> 
  st_join(data_04 |> 
            select(id_area = GEOCODIGO,
                   nombre_area = DESBDT)) |> 
  bind_cols(st_coordinates(tdata_01_sf)) |>
  st_drop_geometry()



In [31]:
glimpse(tdata_01_area)

Rows: 87
Columns: 67
$ CCN                     <chr> "1328020932", "1328000034", "1328000070", "132…
$ CODCNH                  <chr> "281371", "280745", "281124", "281071", "28129…
$ `Nombre Centro`         <chr> "HOSPITAL HM VALLES", "HOSPITAL UNIVERSITARIO …
$ Dirección               <chr> "CALLE SANTIAGO 14", "CARRETERA DE MECO S/N", …
$ Teléfono                <chr> "918883262", "918878100", "917810682", "916219…
$ `Cód. Municipio`        <chr> "280053", "280053", "280072", "280072", "28013…
$ Municipio               <chr> "Alcalá de Henares", "Alcalá de Henares", "Alc…
$ `Cód. Provincia`        <chr> "28", "28", "28", "28", "28", "28", "28", "28"…
$ Provincia               <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRI…
$ `Cód. CCAA`             <chr> "13", "13", "13", "13", "13", "13", "13", "13"…
$ CCAA                    <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRI…
$ `Código Postal`         <chr> "28801", "28805", "28922", "28922", "28300", "…
$ CAMAS            

In [32]:
tdata_01_area |> slice_head(n = 5)

CCN,CODCNH,Nombre Centro,Dirección,Teléfono,Cód. Municipio,Municipio,Cód. Provincia,Provincia,Cód. CCAA,...,SPECT,PET,MAMOS,DO,DIAL,CalidadGeocodificacion,id_area,nombre_area,X,Y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1328020932,281371,HOSPITAL HM VALLES,CALLE SANTIAGO 14,918883262,280053,Alcalá de Henares,28,MADRID,13,...,0,0,1,1,0,PointAddress,03,Este,-3.367372,40.48302
1328000034,280745,HOSPITAL UNIVERSITARIO PRINCIPE DE ASTURIAS,CARRETERA DE MECO S/N,918878100,280053,Alcalá de Henares,28,MADRID,13,...,0,0,2,1,28,Manual,03,Este,-3.347602,40.51025
1328000070,281124,HOSPITAL QUIRONSALUD SUR,CALLE DE ESTAMBUL 30,917810682,280072,Alcorcón,28,MADRID,13,...,0,0,1,1,0,StreetAddress,08,Sur-Oeste I,-3.849035,40.34248
1328000066,281071,HOSPITAL UNIVERSITARIO FUNDACION ALCORCON,CALLE BUDAPEST 1,916219400,280072,Alcorcón,28,MADRID,13,...,0,0,2,1,28,Manual,08,Sur-Oeste I,-3.837394,40.34887
1328000084,281292,HOSPITAL DEL TAJO S.A.,AVENIDA AMAZONAS CENTRAL S/N,918014100,280133,Aranjuez,28,MADRID,13,...,0,0,1,0,13,Manual,11,Sur Ii,-3.610795,40.05726


* Añadir a las listas de espera el código del hospital
* Se mantiene el nombre de la descarga de lista de espera, pero los joins se harán siempre por código

In [33]:
tdata_03_codigos <- tdata_03 |> 
  left_join(tdata_01 |> select(NOMBRE, CODCNH),
            by = c("Hospital" = "NOMBRE")) 

In [34]:
glimpse(tdata_03_codigos)

Rows: 55,680
Columns: 7
$ Hospital          <chr> "HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA A…
$ Especialidad      <chr> "Angiología y Cirugía Vascular", "Angiología y Cirug…
$ total_pacientes   <dbl> 439, 437, 429, 419, 418, NA, 408, 399, 396, 393, 378…
$ media_tiempo_dias <dbl> 56.29, 55.13, 53.99, 56.29, 55.82, NA, 57.45, 57.61,…
$ ano               <dbl> 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022…
$ semana            <dbl> 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, …
$ CODCNH            <chr> "280148", "280148", "280148", "280148", "280148", "2…


In [35]:
tdata_03_codigos |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,media_tiempo_dias,ano,semana,CODCNH
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,439,56.29,2022,44,280148
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,437,55.13,2022,43,280148
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,429,53.99,2022,42,280148
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,419,56.29,2022,41,280148
HOSPITAL CENTRAL DE LA CRUZ ROJA SAN JOSE Y SANTA ADELA,Angiología y Cirugía Vascular,418,55.82,2022,40,280148


* Homogeneizar datos de hospitales

In [36]:
names(tdata_02)

[1] "CCN"                     "CODCNH"                 
 [3] "Nombre Centro"           "Dirección"              
 [5] "Teléfono"                "Cód. Municipio"         
 [7] "Municipio"               "Cód. Provincia"         
 [9] "Provincia"               "Cód. CCAA"              
[11] "CCAA"                    "Código Postal"          
[13] "CAMAS"                   "Cód. Clase de Centro"   
[15] "Clase de Centro"         "Cód. Dep. Funcional"    
[17] "Dependencia Funcional"   "Forma parte Complejo"   
[19] "CODIDCOM"                "Nombre del Complejo"    
[21] "ALTA"                    "Email"                  
[23] "OBJECTID"                "NOMBRE"                 
[25] "DIRECCION"               "TELEFONO"               
[27] "TELEFONO2"               "TELEFAX"                
[29] "CODMU"                   "MUNICIPIOS"             
[31] "CODPROV"                 "PROVINCIAS"             
[33] "CODAUTO"                 "COMUNIDADES"            
[35] "CODPOSTAL"               "CODFI"                  
[37] "FINALIDAD_ASISITENCIAL"  "CODPAT"                 
[39] "DEPENDENCIA_PATRIMONIAL" "CODFU"                  
[41] "DEPENDENCIA_FUNCIONAL"   "ACREDOCENT"             
[43] "ESCOMPLE"                "FORCOMPLE"              
[45] "CERRADO"                 "CAPITAL"                
[47] "CIERREFECH"              "CONCIERTO"              
[49] "EMAIL"                   "TAC"                    
[51] "RM"                      "GAM"                    
[53] "HEM"                     "ASD"                    
[55] "LIT"                     "BCO"                    
[57] "ALI"                     "SPECT"                  
[59] "PET"                     "MAMOS"                  
[61] "DO"                      "DIAL"                   
[63] "X"                       "Y"                      
[65] "CalidadGeocodificacion"

In [37]:
tdata_02_clean <- tdata_02 |>
  left_join(tdata_01_area |> 
              select(CODCNH, id_area, nombre_area),
            by = "CODCNH") |>
    select(CODCNH,
         Hospital = `Nombre Centro`,
         id_area,
         nombre_area,
         cmunicipio = `Cód. Municipio`,
         Municipio,
         CAMAS, 
         Clase = `Clase de Centro`,
         Dependencia = `Dependencia Funcional`,
         TAC:Y)


In [38]:
glimpse(tdata_02_clean)

Rows: 87
Columns: 24
$ CODCNH      <chr> "281371", "280745", "281124", "281071", "281292", "281281"…
$ Hospital    <chr> "HOSPITAL HM VALLES", "HOSPITAL UNIVERSITARIO PRINCIPE DE …
$ id_area     <chr> "03", "03", "08", "08", "11", "01", "06", "06", "06", "11"…
$ nombre_area <chr> "Este", "Este", "Sur-Oeste I", "Sur-Oeste I", "Sur Ii", "S…
$ cmunicipio  <chr> "280053", "280053", "280072", "280072", "280133", "280148"…
$ Municipio   <chr> "Alcalá de Henares", "Alcalá de Henares", "Alcorcón", "Alc…
$ CAMAS       <dbl> 20, 507, 81, 402, 98, 132, 170, 53, 192, 339, 344, 217, 13…
$ Clase       <chr> "Otros Centros con Internamiento", "Hospitales Generales",…
$ Dependencia <chr> "PRIVADOS", "SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNI…
$ TAC         <dbl> 1, 3, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 0, 0, 0, 0…
$ RM          <dbl> 1, 2, 2, 2, 1, 1, 3, 2, 0, 0, 0, 2, 2, 1, 2, 2, 0, 0, 0, 0…
$ GAM         <dbl> 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0…
$ HEM         <dbl>

In [39]:
tdata_02_clean |> slice_head(n = 5)

CODCNH,Hospital,id_area,nombre_area,cmunicipio,Municipio,CAMAS,Clase,Dependencia,TAC,...,LIT,BCO,ALI,SPECT,PET,MAMOS,DO,DIAL,X,Y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
281371,HOSPITAL HM VALLES,03,Este,280053,Alcalá de Henares,20,Otros Centros con Internamiento,PRIVADOS,1,...,0,0,0,0,0,1,1,0,-3.367372,40.48302
280745,HOSPITAL UNIVERSITARIO PRINCIPE DE ASTURIAS,03,Este,280053,Alcalá de Henares,507,Hospitales Generales,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNIDADES AUTÓNOMAS,3,...,0,0,0,0,0,2,1,28,-3.347602,40.51025
281124,HOSPITAL QUIRONSALUD SUR,08,Sur-Oeste I,280072,Alcorcón,81,Hospitales Generales,PRIVADOS,1,...,0,0,0,0,0,1,1,0,-3.849035,40.34248
281071,HOSPITAL UNIVERSITARIO FUNDACION ALCORCON,08,Sur-Oeste I,280072,Alcorcón,402,Hospitales Generales,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNIDADES AUTÓNOMAS,2,...,1,0,0,0,0,2,1,28,-3.837394,40.34887
281292,HOSPITAL DEL TAJO S.A.,11,Sur Ii,280133,Aranjuez,98,Hospitales Generales,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNIDADES AUTÓNOMAS,1,...,0,0,0,0,0,1,0,13,-3.610795,40.05726


## Synthetic Data Generation

No aplica

## Fake Data Generation

No aplica

## Open Data

No aplica

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar las cadenas añadida del fichero para diferenciarlas

1. Datos de hospitales

<font color='tomato'> Identificamos los datos a guardar</font>

In [40]:
data_to_save_01 <- tdata_02_clean


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [41]:
caso <- "CU_25"
proceso <- '_05'
tarea <- "_05"
archivo <- "_01"
proper <- "_hospitales"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [42]:
# file_save_01 <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out_01 <- paste0(oPath, file_save_01)
# write_csv(data_to_save_01, path_out_01)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [43]:
file_save_01 <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out_01 <- paste0(oPath, file_save_01)
write_csv(data_to_save_01, path_out_01)

cat('File saved as: ')
path_out_01

File saved as: 

[1] "Data/Output/CU_25_05_05_01_hospitales.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [44]:
path_in_01 <- paste0(iPath, file_save_01)
file.copy(path_out_01, path_in_01, overwrite = TRUE)

[1] TRUE

2. Lista de espera (actualizada con código de hospital)

<font color='tomato'> Identificamos los datos a guardar</font>

In [45]:
data_to_save_02 <- tdata_03_codigos


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [46]:
archivo <- "_02"
proper <- "_lista_espera"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [47]:
# file_save_02 <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out_02 <- paste0(oPath, file_save_02)
# write_csv(data_to_save_02, path_out_02)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [48]:
file_save_02 <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out_02 <- paste0(oPath, file_save_02)
write_csv(data_to_save_02, path_out_02)

cat('File saved as: ')
path_out_02

File saved as: 

[1] "Data/Output/CU_25_05_05_02_lista_espera.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [49]:
path_in_02 <- paste0(iPath, file_save_02)
file.copy(path_out_02, path_in_02, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

Para que funcione este código se necesita:

* Las rutas de archivos `Data/Input` y `Data/Output` deben existir (relativas a la ruta del _notebook_)
* El paquete tcltk instalado para seleccionar archivos interactivamente. No se necesita en producción.
* Los paquetes tcltk, sf, readr, dplyr, stringr, readxl deben estar instalados.


### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:
* R 4.2.2
* tcltk 4.2.2
* sf 1.0.9
* readr 2.1.3
* dplyr 1.0.10
* stringr 1.5.0
* readxl 1.4.1

### Data structures

#### Objeto `tdata_02_clean`

- Los datos de origen son del catálogo nacional de hospitales a los que se ha añadido las coordenadas y el área de salud
- Hay 87 filas con información de las siguientes variables:
	* CODCNH
	* Hospital
	* id_area
	* nombre_area
	* cmunicipio
	* Municipio
	* CAMAS
	* Clase
	* Dependencia
	* TAC
	* RM
	* GAM
	* HEM
	* ASD
	* LIT
	* BCO
	* ALI
	* SPECT
	* PET
	* MAMOS
	* DO
	* DIAL
	* X
	* Y


#### Objeto `tdata_03_codigos`

- Son los datos de la lista de espera a los que se ha añadido el código de hospital
- Hay 55680 filas con información de las siguientes variables:
	* Hospital
	* Especialidad
	* total_pacientes
	* media_tiempo_dias
	* ano
	* semana
	* CODCNH


#### Observaciones generales sobre los datos

* En el fichero de lista de espera se mantiene el nombre de hospital de la descarga
* Pero se añade el código oficial, por el que se harán las uniones. El nombre oficial se tomaría del catálogo nacional
* Si se quisieran usar otras agrupaciones de zonas bastaría cambiar el fichero json

### Consideraciones para despliegue en piloto

* No aplica

### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>

- Se han homogeneizado nombres de hospitales
- Se han asignado coordenadas al catálogo nacional de hospitales
- Se han asignado áreas de salud a los hospitales
- Se han añadido los códigos de hospitales a las listas de espera


#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se deben añadir indicadores a las áreas de salud para usar en los modelos

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>

In [50]:
# incluir código